# 加载多种格式文件

## 准备

In [ ]:
%%time
%%capture

!pip install openpyxl
!pip install docx2txt

In [21]:
%%time

from llama_index.core import SummaryIndex, Document
from llama_index.core import Settings

from llama_index.llms.openai_like import OpenAILike
from llama_index.embeddings.ollama import OllamaEmbedding

from milvus import default_server
from pymilvus import connections, utility

from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.core import (
    SimpleDirectoryReader,
    load_index_from_storage,
    VectorStoreIndex,
    StorageContext,
)
from llama_index.vector_stores.milvus import MilvusVectorStore

from llama_index.core.llms import ChatMessage

from IPython.display import display

CPU times: user 50 µs, sys: 0 ns, total: 50 µs
Wall time: 52.2 µs


## pdf

### 文字版本

In [4]:
%%time
%%capture

# load documents
documents = SimpleDirectoryReader(input_files=["books2/第三次浪潮.pdf"]).load_data()

CPU times: user 19.3 s, sys: 7.98 ms, total: 19.3 s
Wall time: 19.3 s


In [6]:
documents[0].metadata

{'page_label': '1',
 'file_name': '第三次浪潮.pdf',
 'file_path': 'books2/第三次浪潮.pdf',
 'file_type': 'application/pdf',
 'file_size': 27212461,
 'creation_date': '2024-05-10',
 'last_modified_date': '2024-05-10'}

### 扫描版

#### NougatOCR

In [ ]:
自带的功能，使用NougatOCR，meta开源的工具，仅适用于英文科技论文。

无法支持中文。

In [12]:
%%time
%%capture

!pip install llama-index-readers-nougat-ocr

CPU times: user 64.5 ms, sys: 16.2 ms, total: 80.7 ms
Wall time: 17.6 s


In [ ]:
%%time

# load documents
# documents = SimpleDirectoryReader(input_files=["books2/人工智能的未来.pdf"]).load_data()

from pathlib import Path
from llama_index.readers.nougat_ocr import PDFNougatOCR

reader = PDFNougatOCR()
pdf_path = Path("books2/人工智能的未来.pdf")
documents = reader.load_data(pdf_path)

#### 国内ocr方案

国内相关方案：

- https://github.com/RapidAI/RapidOCR - chatchat在用，只针对文字，排版和图片貌似不支持
- paddle，百度飞桨，cnn库找不到，兼容性可能有问题

目前结论是放弃实现这个功能。

将来使用合作伙伴的。

## cvs

In [15]:
import numpy as np
import pandas as pd

# 加载 CSV 文件
df = pd.read_csv('./data/test-100.csv')
# 显示前10条
df.head(10)

,text1,text2
0,本文总结了十个可穿戴产品的设计原则，而这些原则，同样也是笔者认为是这个行业最吸引人的地方：1...,可穿戴技术十大设计原则
1,2007年乔布斯向人们展示iPhone并宣称“它将会改变世界”，还有人认为他在夸大其词，然而...,经济学人：智能手机将成为“真正的个人电脑”
2,雅虎发布2014年第四季度财报，并推出了免税方式剥离其持有的阿里巴巴集团15％股权的计划，打...,雅虎宣布剥离阿里巴巴股份
3,2014年，51信用卡管家跟宜信等P2P公司合作，推出线上信贷产品“瞬时贷”，其是一种纯在线...,51信用卡管家，预计2015年放贷额度远超30亿
4,目前世界上有着几百种编程语言，我应该学哪个?如何选择“正确”的编程语言进行学习?我所学的语言...,如何选择正确的编程语言进行学习
5,受众在哪里，媒体就应该在哪里，媒体的体制、内容、技术就应该向哪里转变。媒体融合关键是以人为本...,媒体融合关键是以人为本
6,中国铁路总公司消息，自2015年1月5日起，自行车不能进站乘车了。骑友大呼难以接受。这部分希...,自行车不能带上火车
7,今天有传在北京某小区，一光头明星因吸毒被捕的消息。下午北京警方官方微博发布声明通报情况，证实...,北京警方确认李代沫吸毒被捕(图)
8,党的十八大以来，以习近平同志为总书记的党中央高瞻远瞩战略谋划，着力创新发展理念，大力建设生态...,生态兴则文明兴生态衰则文明衰
9,昨晚，中联航空成都飞北京一架航班被发现有多人吸烟。后因天气原因，飞机备降太原机场。几名乘客在...,成都飞北京航班多人吸烟机组人员与未吸烟乘客冲突


In [16]:
%%time
%%capture

# load documents
documents = SimpleDirectoryReader(input_files=["./data/test-100.csv"]).load_data()

CPU times: user 7.23 ms, sys: 0 ns, total: 7.23 ms
Wall time: 7.2 ms


In [18]:
len(documents)

1

In [17]:
documents[0].metadata

{'file_path': 'data/test-100.csv',
 'file_name': 'test-100.csv',
 'file_type': 'text/csv',
 'file_size': 36339,
 'creation_date': '2024-05-11',
 'last_modified_date': '2024-05-11'}

In [20]:
documents[0].text

'本文总结了十个可穿戴产品的设计原则，而这些原则，同样也是笔者认为是这个行业最吸引人的地方：1.为人们解决重复性问题；2.从人开始，而不是从机器开始；3.要引起注意，但不要刻意；4.提升用户能力，而不是取代人, 可穿戴技术十大设计原则\n2007年乔布斯向人们展示iPhone并宣称“它将会改变世界”，还有人认为他在夸大其词，然而在8年后，以iPhone为代表的触屏智能手机已经席卷全球各个角落。未来，智能手机将会成为“真正的个人电脑”，为人类发展做出更大的贡献。, 经济学人：智能手机将成为“真正的个人电脑”\n雅虎发布2014年第四季度财报，并推出了免税方式剥离其持有的阿里巴巴集团15％股权的计划，打算将这一价值约400亿美元的宝贵投资分配给股东。截止发稿前，雅虎股价上涨了大约7％，至51.45美元。, 雅虎宣布剥离阿里巴巴股份\n2014年，51信用卡管家跟宜信等P2P公司合作，推出线上信贷产品“瞬时贷”，其是一种纯在线操作的信贷模式。51信用卡管家创始人孙海涛说，51目前每天放贷1000万，预计2015年，自营产品加上瞬时贷，放贷额度将远超30亿。, 51信用卡管家，预计2015年放贷额度远超30亿\n目前世界上有着几百种编程语言，我应该学哪个?如何选择“正确”的编程语言进行学习?我所学的语言日后能否成为我获取好生活的保障?在这个问题上，很多人都曾经给出了他们都看法。但在我看来，这个问题答案其实非常简单：那就是JavaScript。, 如何选择正确的编程语言进行学习\n受众在哪里，媒体就应该在哪里，媒体的体制、内容、技术就应该向哪里转变。媒体融合关键是以人为本，即满足大众的信息需求，为受众提供更优质的服务。这就要求媒体在融合发展的过程中，既注重技术创新，又注重用户体验。, 媒体融合关键是以人为本\n中国铁路总公司消息，自2015年1月5日起，自行车不能进站乘车了。骑友大呼难以接受。这部分希望下车就能踏上骑游旅程的旅客，只能先办理托运业务，可咨询12306客服电话，就近提前办理。运费每公斤价格根据运输里程不同而不同。, 自行车不能带上火车\n今天有传在北京某小区，一光头明星因吸毒被捕的消息。下午北京警方官方微博发布声明通报情况，证实该明星为李代沫。李代沫伙同另外6人，于17日晚在北京朝阳区三里屯某小区的暂住地内吸食毒品，6人全部被警方抓获，且当事人对犯案实施供认不讳

## excel

没弄出来。

应该能走通，时间原因不弄了。

基本思路是：

- pd加载excel
- pd的dataframe，生成document，再分片形成node
- node就可以加入向量库

In [26]:
df = pd.read_excel('./data/电池比较表.xlsx')
df.head(10)

,序号,电池名称,电压v,容量mAh,能量Wh,总能量Wh,价格,备注
0,1,Apple Magic Mouse2自带电池,3.76,1986,7.46736,7.46736,NaN,输出标称7.28Wh
1,2,南孚锂可充,3.70,750,2.77500,5.55000,NaN,NaN
2,3,菲尼克斯5号充电锂电池L14-1600U,1.50,1600,2.40000,4.80000,NaN,NaN
3,4,金特力5号充电锂电池,3.70,760,2.81200,5.62400,NaN,NaN
4,5,硕而博5号充电锂电池,1.50,1200,1.80000,3.60000,NaN,NaN
5,6,力特朗3.2v充电磷酸铁锂电池,3.20,1200,3.84000,7.68000,NaN,疑问，2只是否能用于鼠标


In [45]:
from llama_index.core import Document

documents = [
    Document(
        content=row['电池名称'],
        metadata={
            'ID': row['序号'],
            'Name': row['电池名称'],
            'Voltage': row['电压v']
        }
    )
    for _, row in df.iterrows()
]

documents

[Document(id_='94e34b2d-994d-4946-9652-51b3398dec5e', embedding=None, metadata={'ID': 1, 'Name': 'Apple Magic Mouse2自带电池', 'Voltage': 3.76}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 Document(id_='002ad0ea-446e-4fc0-87ff-e840d8a82bab', embedding=None, metadata={'ID': 2, 'Name': '南孚锂可充', 'Voltage': 3.7}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 Document(id_='6a57e3f8-055a-41a2-a1c1-9f79707d8a50', embedding=None, metadata={'ID': 3, 'Name': '菲尼克斯5号充电锂电池L14-1600U', 'Voltage': 1.5}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='', start_char_idx=None, end_char_i

In [46]:
from llama_index.core.text_splitter import SentenceSplitter

splitter = SentenceSplitter(
    chunk_size=200,
    chunk_overlap=0,
    paragraph_separator="\n\n"
)
nodes = splitter.get_nodes_from_documents(documents)

len(nodes)

6

In [38]:
nodes[0]

TextNode(id_='9f9497c5-16fa-49b8-8bff-4e9a00da2747', embedding=None, metadata={'ID': 1, 'Name': 'Apple Magic Mouse2自带电池', 'Voltage': 3.76}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='f405fff2-3360-4213-9ccd-8df71565e69e', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'ID': 1, 'Name': 'Apple Magic Mouse2自带电池', 'Voltage': 3.76}, hash='b70bd8013142359d964997d6edd74a159c129fc4ea47b2374a131f3bac8ea6fd')}, text='', start_char_idx=0, end_char_idx=0, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n')

## docx

In [49]:
%%time
%%capture

# load documents
documents = SimpleDirectoryReader(input_files=["data/光明云媒简介.docx"]).load_data()

CPU times: user 6.02 ms, sys: 40 µs, total: 6.06 ms
Wall time: 5.21 ms


In [50]:
len(documents)

1

In [53]:
documents[0].metadata

{'file_name': '光明云媒简介.docx',
 'file_path': 'data/光明云媒简介.docx',
 'file_type': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document',
 'file_size': 16341,
 'creation_date': '2024-05-11',
 'last_modified_date': '2024-05-11'}

In [54]:
documents[0].text

'光明云媒简介\n\n\n\n公司简介：\n\n光明日报创刊于1949年6月16日，是中共中央主办，以知识分子为主要读者对象的思想文化大报；毛泽东、周恩来、朱德等老一辈无产阶级革命家为光明日报题词。\n\n与真理同行，与时代同步。1978年5月11日，光明日报特约评论员文章《实践是检验真理的唯一标准》，对中国社会发展产生深远影响。成为中国进入新时期的标志事件。2005年6月21日，时任浙江省委书记的习近平同志在光明日报发表文章《弘扬“红船精神”走在时代前列》，首次提出并阐释了“红船精神”这一中国共产党的源头精神。\n\n光明日报在全国设有37个记者站，在世界23个国家和地区派有常驻记者，读者遍及社会各界和120多个国家和地区，报纸发行量近千万份。\n\n光明网创立于1998年1月1日，是中央网络安全和信息化领导小组办公室（网信办） 主管，光明日报社主办，的中央重点新闻网站。二十余年来，光明延伸和发挥了光明日报的核心价值和传统优势。在文化届、知识界广大读者和网友中有着强大影响力。定位于知识分子网上精神家园、权威思想理论文化网站。“新闻视野，文化视角，思想深度，理论高度”。 \n\n每逢国家重大会议、事件，光明日报、光明网都进行会实时直播，包括二十大专题策划，两会等。日常还有多档直播栏目，例如：核心价值观百场讲坛，中华地名文化直播，致非遗敬匠心直播活动。多方位地将党政重要事件、文化、科普内容带给用户。\n\n光明云媒（上海）文化传播有限公司成立于2013年，隶属光明网。是光明日报，光明网在新媒体领域的延伸。定位于新媒体领域内容及平台运营服务提供商。公司以光明网“新闻视野，文化视角，思想深度，理论高度”的理念为指导准则，结合自身的自制能力，将党政、文化、科教类等优势内容通过移动终端向用户提供服务。\n\n光明云媒是中国移动集团咪咕视频战略合作伙伴，长期为咪咕视频提供优质的文化，科普，纪实类内容。与北京市文旅局、北京市西城区非遗中心进行非遗内容的策划、拍摄、制作，并在新媒体多渠道上宣传推广。\n\n光明云媒凭借自身技术、平台及多年新媒体运营经验优势，与退役军人事务部退役军人培训中心，国家市场监管总局培训中心、国家海关总署媒体中心、国家文物局等多家部委机关开展在线上培训、平台运营等全方位服务的紧密合作。\n\n\n\n\n\n二、平台开发运维有关案例\n\n光明日报全媒体中心

## doc

In [ ]:
doc 目前看是无效的。不支持。

In [56]:
%%time
%%capture

# load documents
documents = SimpleDirectoryReader(input_files=["data/光明云媒简介.doc"]).load_data()

len(documents)

CPU times: user 2.12 ms, sys: 43 µs, total: 2.16 ms
Wall time: 2.07 ms


## pptx

不可用。

cohere transformer相关库版本有问题，报错。

In [ ]:
%%time
%%capture

!pip install torch transformers python-pptx Pillow

In [ ]:
%%time

import os

os.environ['http_proxy'] = "http://192.168.0.134:7890"
os.environ['https_proxy'] = "http://192.168.0.134:7890"

# load documents
documents = SimpleDirectoryReader(input_files=["data/低代码方案改变软件实现模式.pptx"]).load_data() # 报错，用最新transformers没有cohere模块

len(documents)

## epub

不可用。

有警告，最终没有生成documents。

In [65]:
%%time
%%capture

!pip install EbookLib html2text

CPU times: user 6.92 ms, sys: 17.1 ms, total: 24 ms
Wall time: 4.22 s


In [66]:
# load documents
documents = SimpleDirectoryReader(input_files=["data/中国近代史.epub"]).load_data()

len(documents)

Failed to load file data/中国近代史.epub with error: 'NoneType' object has no attribute 'nsmap'. Skipping...


/usr/local/lib/python3.10/dist-packages/ebooklib/epub.py:1423: FutureWarning: This search incorrectly ignores the root element, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/xmlns:rootfile[@media-type]'
  for root_file in tree.findall('//xmlns:rootfile[@media-type]', namespaces={'xmlns': NAMESPACES['CONTAINERNS']}):


0

## markdow

In [67]:
# load documents
documents = SimpleDirectoryReader(input_files=["data/obsidian 使用总结.md"]).load_data()

len(documents)

12

In [68]:
documents[0].metadata

{'file_path': 'data/obsidian 使用总结.md',
 'file_name': 'obsidian 使用总结.md',
 'file_type': 'text/markdown',
 'file_size': 2916,
 'creation_date': '2024-05-11',
 'last_modified_date': '2024-05-11'}

In [72]:
documents[6].text

'\n\niCloud\n\n[[iCloud]]\n\n- 使用 iCloud, iPhone/iPad/macOS 之间同步\n- 需要\n\t- 先在 iOS 设备上创建 iCloud vault，设置为 store in iCloud\n\t- 然后在 macOS 的 iCloud/obsidian/xxx 打开\n\t- 如果在 iOS 设备创建新的文件，可能需要在 macOS finder "现在下载"\n\t\t- macOS 的 obsidian App 不能读取未下载的文件，即使在 finder 中可见\n - 问题\n\t -  不能端到端加密\n  \n'

## 图片

### png 文本

In [73]:
# load documents
documents = SimpleDirectoryReader(input_files=["data/全球资源-序.PNG"]).load_data()

len(documents)

1

In [74]:
documents[0].metadata

{'file_path': 'data/全球资源-序.PNG',
 'file_name': '全球资源-序.PNG',
 'file_type': 'image/png',
 'file_size': 307766,
 'creation_date': '2024-05-11',
 'last_modified_date': '2024-05-11'}

In [75]:
documents[0].text

''

### jpg 人像

In [76]:
# load documents
documents = SimpleDirectoryReader(input_files=["data/刘德华.jpg"]).load_data()

len(documents)

1

In [77]:
documents[0].metadata

{'file_path': 'data/刘德华.jpg',
 'file_name': '刘德华.jpg',
 'file_type': 'image/jpeg',
 'file_size': 408245,
 'creation_date': '2024-05-11',
 'last_modified_date': '2024-05-11'}

In [78]:
documents[0].text

''